# *SQL e Python no Mundo do Futebol e suas possíveis aplicações*

## <u>Com Python...<u/>

Para o  Algoritmo SVM simulei uma situação onde conto com um dataset com diversas informações classificando as pessoas que recebem  mais ou menos que 25 mil reais anuais, sendo essas pessoas, sócios torcedores de um time x. O objetivo é funcionar como uma empresa que vende relatórios de inteligência para que possam ser aproveitados pelos clubes de futebol.

##### O que é SVM (Support Vector Machine) ?

Basicamente, o SVM é um algoritmo supervisionado que tenta criar uma linha (ou uma fronteira) que melhor separa os dados.

![](Images/svm1.JPG)

Essa linha vermelha é o melhor Hiperplano que o algoritmo conseguiu criar, ou seja, o hiperplano que melhor separa as classes.
Perceba que temos mais duas linhas pontilhadas que estão próximas as estrelas, estes também são hiperplanos criados pelo algoritmo.
<br>
E por que dizemos que a linha do meio é o melhor Hiperplano?
<br>
Porque esta tem uma maior distância entre as classes, sendo assim uma linha mais confiável! Esta linha é a que separa as classes com uma certa folga.

##### Como escolher o melhor hiperplano?

O algoritmo calcula os vetores de suporte para então escolher o melhor hiperplanos (por isso o nome Support Vectors Machine). Os vetores de suporte são pontos que estão mais próximos das linhas, e a distância entre esses pontos são chamadas de Margens. Com essa informação o algoritmo escolhe um hiperplano que tem uma margem maior, por isso o hiperplano do meio é considerado o melhor.

![](Images/svm2.JPG)

O SVM escolhe um Hiperplano com uma maior margem para ser um um modelo mais robusto. Mas essa configuração pode ser alterada. O SVM possui um parâmetro C que basicamente dita como será construída essa margem que separa as classes. Valores maiores para esse parâmetro diz para o SVM ser mais tolerante a erros, assim o modelo irá tentar escolher hiperplanos com margens menores. Um valor menor do parâmetro C diz para o SVM ser menos tolerante a erros,isso significa que o modelo irá escolher hiperplanos com margens maiores.

#### Importando todas as bibliotecas necessárias

In [2]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from math import sqrt
import numpy as np
import pandas as pd
import joblib

#### Leitura dos dataframes

In [3]:
df = pd.read_csv('adult2.csv',names=[
        "Idade", "Classe de Trabalho", "Peso Final", "Educacao", "Id_Educacao", "Estado Civil",
        "Ocupacao", "Relacao", "Raca", "Sexo", "Ganho de Capital", "Perda de Capital",
        "Horas-Por-Semana", "Nacionalidade", "Ganho Anual"], # Traduzindo colunas para o português
        sep=r'\s*;\s*',
        engine='python',
        na_values="?", skiprows = 1).dropna() # Linha de código feita para excluir linhas que contém dados faltantes
df

,Idade,Classe de Trabalho,Peso Final,Educacao,Id_Educacao,Estado Civil,Ocupacao,Relacao,Raca,Sexo,Ganho de Capital,Perda de Capital,Horas-Por-Semana,Nacionalidade,Ganho Anual
0,50,Trabalhador-nao-autonomo,83311,Bacharelado,13,Conjuge-civ-casado,Executivo-gerencial,Marido,Branco,Masculino,0,0,13,Brasil,<=25K
1,38,Privado,215646,Formado-EM,9,Divorciado,Limpeza,Nao-membro-da-familia,Branco,Masculino,0,0,40,Brasil,<=25K
2,53,Privado,234721,2-colegial,7,Conjuge-civ-casado,Limpeza,Marido,Negro,Masculino,0,0,40,Brasil,<=25K
3,28,Privado,338409,Bacharelado,13,Conjuge-civ-casado,Profissional-especializado,Esposa,Negro,Feminino,0,0,40,Brasil,<=25K
4,37,Privado,284582,Mestrado,14,Conjuge-civ-casado,Executivo-gerencial,Esposa,Branco,Feminino,0,0,40,Brasil,<=25K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,Privado,257302,Associacao-academica,12,Conjuge-civ-casado,Suporte-Tecnico,Esposa,Branco,Feminino,0,0,38,Brasil,<=25K
32556,40,Privado,154374,Formado-EM,9,Conjuge-civ-casado,Operador-maquina,Marido,Branco,Masculino,0,0,40,Brasil,>25K
32557,58,Privado,151910,Formado-EM,9,Viuvo,Assistente-administrativo,Nao-casado,Branco,Feminino,0,0,40,Brasil,<=25K
32558,22,Privado,201490,Formado-EM,9,Nunca-casou,Assistente-administrativo,Filho-proprio,Branco,Masculino,0,0,20,Brasil,<=25K


#### Substituindo df por df_novo

In [4]:
df_novo = df

### Transformando as 'str' do dataset em números para o treinamento dos algoritmos

In [5]:
# Função 'embar_atrib'
def embar_atrib(atributo, alvo):
    dic_em_branco = {}
    localTargetData = {}
    for index, element in enumerate(atributo):
        if element not in localTargetData:
            if alvo[index] == '<=25K':
                localTargetData[element] = [1,0]
            else:
                localTargetData[element] = [0,1]
        else:
            if alvo[index] == '<=25K':
                localTargetData[element][0] +=1
            else:
                localTargetData[element][1] += 1
    return localTargetData

In [6]:
# Laço for responsável por fazer as trocas de 'str' por 'float' nos df's
for i in (1,3,5,6,7,8,9,13):
    replaceDict = embar_atrib(df_novo.iloc[:,i:i+1].values.transpose()[0].tolist(),df_novo.iloc[:,df_novo.shape[1]-1:df_novo.shape[1]].values.transpose()[0].tolist())
    for key in replaceDict:
        df_novo = df_novo.replace(key, replaceDict[key][1]/(replaceDict[key][1]+replaceDict[key][0]))
        print('Trocando {0} por {1}'.format(key, replaceDict[key][1]/(replaceDict[key][1]+replaceDict[key][0]) ))

Trocando Trabalhador-nao-autonomo por 0.2849271940181031
Trocando Privado por 0.21867289390200917
Trocando Chefe-de-estado por 0.27216653816499614
Trocando Governo-federal por 0.38645833333333335
Trocando Governo-municipal por 0.29479216435738176
Trocando Trabalhador-autonomo por 0.557347670250896
Trocando Sem-remuneracao por 0.0
Trocando Bacharelado por 0.4199961397413627
Trocando Formado-EM por 0.16342295345104335
Trocando 2-colegial por 0.056818181818181816
Trocando Mestrado por 0.5617910447761194
Trocando 9-ano por 0.056155507559395246
Trocando Faculdade por 0.19955719557195573
Trocando Associacao-academica por 0.25392156862745097
Trocando Curso-profissionalizante por 0.2634367903103709
Trocando 7ou8-ano por 0.06631762652705062
Trocando Doutorado por 0.7412060301507538
Trocando Pos-graduado por 0.7437275985663082
Trocando 5ou6-ano por 0.0462046204620462
Trocando 1-colegial por 0.07220216606498195
Trocando Pre-escola por 0.0
Trocando 3-colegial por 0.07888040712468193
Trocando 1ou4-

#### Substituindo as classes finais (>25K por 1; <=25K por 0) 

In [7]:
df_novo = df_novo.replace('>25K', 1)
df_novo = df_novo.replace('<=25K',0)

### Novo dataset

In [8]:
df_novo

,Idade,Classe de Trabalho,Peso Final,Educacao,Id_Educacao,Estado Civil,Ocupacao,Relacao,Raca,Sexo,Ganho de Capital,Perda de Capital,Horas-Por-Semana,Nacionalidade,Ganho Anual
0,50,0.284927,83311,0.419996,13,0.454774,0.484014,0.455683,0.264030,0.313802,0,0,13,0.249048,0
1,38,0.218673,215646,0.163423,9,0.107327,0.062774,0.106562,0.264030,0.313802,0,0,40,0.249048,0
2,53,0.218673,234721,0.056818,7,0.454774,0.062774,0.455683,0.129942,0.313802,0,0,40,0.249048,0
3,28,0.218673,338409,0.419996,13,0.454774,0.449034,0.491289,0.129942,0.113494,0,0,40,0.249048,0
4,37,0.218673,284582,0.561791,14,0.454774,0.484014,0.491289,0.264030,0.113494,0,0,40,0.249048,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,0.218673,257302,0.253922,12,0.454774,0.304957,0.491289,0.264030,0.113494,0,0,38,0.249048,0
32556,40,0.218673,154374,0.163423,9,0.454774,0.124875,0.455683,0.264030,0.313802,0,0,40,0.249048,1
32557,58,0.218673,151910,0.163423,9,0.096429,0.134518,0.066035,0.264030,0.113494,0,0,40,0.249048,0
32558,22,0.218673,201490,0.163423,9,0.048633,0.134518,0.014586,0.264030,0.313802,0,0,20,0.249048,0


### Aplicando o algoritmo SVM

In [9]:
# Atributos a serem considerados
X = np.array(df_novo.drop('Ganho Anual',1))
X

array([[5.00000000e+01, 2.84927194e-01, 8.33110000e+04, ...,
        0.00000000e+00, 1.30000000e+01, 2.49047759e-01],
       [3.80000000e+01, 2.18672894e-01, 2.15646000e+05, ...,
        0.00000000e+00, 4.00000000e+01, 2.49047759e-01],
       [5.30000000e+01, 2.18672894e-01, 2.34721000e+05, ...,
        0.00000000e+00, 4.00000000e+01, 2.49047759e-01],
       ...,
       [5.80000000e+01, 2.18672894e-01, 1.51910000e+05, ...,
        0.00000000e+00, 4.00000000e+01, 2.49047759e-01],
       [2.20000000e+01, 2.18672894e-01, 2.01490000e+05, ...,
        0.00000000e+00, 2.00000000e+01, 2.49047759e-01],
       [5.20000000e+01, 5.57347670e-01, 2.87927000e+05, ...,
        0.00000000e+00, 4.00000000e+01, 2.49047759e-01]])

In [10]:
# Classe Final
y = np.array(df_novo['Ganho Anual'])
y

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [11]:
# Separando os X's e Y's para teste e treino (30% dos dados serão utilizados para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [12]:
# Algoritmo SVM
svm = svm.SVC(C=1.0) # O parâmetro "C"

In [13]:
# Normalização dos valores X_train e y_train com a função 'fit()'
svm.fit(X_train, y_train)

SVC()

In [14]:
# y_pred = valores preditos
y_pred = svm.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [15]:
# y_test = valores testados
y_test

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### Avaliando o classificador SVM

In [16]:
print(classification_report(y_test, y_pred, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.78      1.00      0.88      6972
           1       0.98      0.15      0.26      2244

    accuracy                           0.79      9216
   macro avg       0.88      0.57      0.57      9216
weighted avg       0.83      0.79      0.73      9216



![](Images/metricas.JPG)

In [17]:
confusion_matrix(y_test, y_pred)

array([[6966,    6],
       [1911,  333]], dtype=int64)

![](Images/Matriz1.JPG)
![](Images/Matriz2.JPG)

Das 8877 pessoas da amostra que ganham menos de 25K, 6966 foram previstas corretamente, enquanto 1911 foram preditas ganhando mais de 25K. Já das 339 pessoas da amostra que ganham mais de 25K, 333 foram previstas corretamente, enquanto 6 foram preditas ganhando menos de 25K.

### Persistindo o algoritmo SVM com o joblib

In [18]:
arq_svm = 'SVM_SocioTorcedor.sav'
joblib.dump(svm,arq_svm)

['SVM_SocioTorcedor.sav']

In [19]:
svm_carregado = joblib.load(arq_svm)
resultado = svm_carregado.score(X_test, y_test)
print(resultado)

0.7919921875


#### ACURÁCIA FINAL = 79,1%